In [133]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [29]:
data1 = pd.read_csv('http://api.bitcoincharts.com/v1/csv/btcalphaUSD.csv.gz',header = None)
data1.head()

,0,1,2
0,1478025745,715.000,0.002797
1,1478031460,720.000,0.001300
2,1478031491,720.199,0.001264
3,1478033362,720.199,0.002902
4,1478033362,720.200,0.001223


In [30]:
data1['date'] = pd.to_datetime(data1[0],unit='s')
data1.columns= ['timestamp','price','volume','date']
data1  = data1[data1.timestamp>=1512950400]# after dec 24, 2017

In [31]:
data1.index = data1.date
bars = data1.price.resample('60min', how='ohlc')
volumes = data1.volume.resample('60min', how='sum')
data1 = pd.concat([bars,volumes],axis=1)
data1['date'] = data1.index

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).ohlc()
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
data1.head()

,open,high,low,close,volume,date,ts
date,,,,,,,
2017-12-11 00:00:00,14938.0,15365.0,14938.0,15283.0,18.205027,2017-12-11 00:00:00,1512930600
2017-12-11 01:00:00,15286.0,15967.0,15286.0,15943.0,10.753708,2017-12-11 01:00:00,1512934200
2017-12-11 02:00:00,15996.0,16099.0,15717.0,16099.0,17.489776,2017-12-11 02:00:00,1512937800
2017-12-11 03:00:00,16099.0,16580.0,15797.0,15797.0,18.776254,2017-12-11 03:00:00,1512941400
2017-12-11 04:00:00,16200.0,16510.0,15820.0,16034.0,23.037675,2017-12-11 04:00:00,1512945000


In [33]:
data2 = pd.read_csv('https://finance.google.com/finance/getprices?p=60d&i=3601&f=d,o,h,l,c,v&q=SP600',
                    skiprows = 7,header = None)
data2.shape

(350, 6)

In [34]:
data2.columns =['date','open','close','high','low','vol']

In [35]:
data2['date'] = data2['date'].str[1:]

In [36]:
data2['date'] = pd.to_datetime(data2['date'],unit='s')

In [37]:
 data1['ts']=data1[['date']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [38]:
data2['ts'] =data2[['date']].apply(lambda x: x[0].timestamp(), axis=1).astype(int)

In [39]:
final = data1.merge(data2 , on = 'ts')


In [51]:
final = final.drop(['date_x','ts','date_y'],axis=1)

In [63]:
corr_mat = final.corr()

In [80]:
corr_mat=corr_mat.iloc[0:5,5:10]

In [84]:
t = [0,0.5,0.2,0.2,0.1]*corr_mat.sum(axis=1)


In [87]:
t

open_x     0.000000
high_x     0.237120
low_x      0.093688
close_x    0.093135
volume    -0.016831
dtype: float64

In [112]:
final['trade_pos/neg'] = np.zeros((350,1))

In [115]:
final['trade_pos/neg'] =np.where(final.close_y>final.open_y,1,0)

In [118]:
final_data =final.drop(['open_y','close_y','high_y','low_y','vol'],axis=1)

In [122]:
X_train,X_test,y_train,y_test = train_test_split(final_data.drop(['trade_pos/neg'],axis=1),final_data['trade_pos/neg'],
                                                 test_size=0.05, random_state=42)

# Xgboost 

In [152]:
xg_train = xgb.DMatrix(X_train,label =y_train)
xg_test = xgb.DMatrix(X_test,label=y_test)
param ={}

param['objective'] = 'multi:softmax'  #  'multi:softprob' for predict probability
param['eta'] = 0.1
param['max_depth'] = 3
param['silent'] = 0
param['nthread'] = 4


param['num_class'] = 2
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 500

bst = xgb.train(param, xg_train, num_round, watchlist)
pred = bst.predict(xg_test)
error = np.sum(pred != y_test) / y_test.shape[0]
print('Test error = {}'.format(error))

[0]	train-merror:0.060241	test-merror:0
[1]	train-merror:0.060241	test-merror:0
[2]	train-merror:0.060241	test-merror:0
[3]	train-merror:0.060241	test-merror:0
[4]	train-merror:0.060241	test-merror:0
[5]	train-merror:0.060241	test-merror:0
[6]	train-merror:0.060241	test-merror:0
[7]	train-merror:0.060241	test-merror:0
[8]	train-merror:0.060241	test-merror:0
[9]	train-merror:0.060241	test-merror:0
[10]	train-merror:0.057229	test-merror:0
[11]	train-merror:0.060241	test-merror:0
[12]	train-merror:0.057229	test-merror:0
[13]	train-merror:0.057229	test-merror:0
[14]	train-merror:0.057229	test-merror:0
[15]	train-merror:0.057229	test-merror:0
[16]	train-merror:0.057229	test-merror:0
[17]	train-merror:0.057229	test-merror:0
[18]	train-merror:0.057229	test-merror:0
[19]	train-merror:0.057229	test-merror:0
[20]	train-merror:0.057229	test-merror:0
[21]	train-merror:0.057229	test-merror:0
[22]	train-merror:0.057229	test-merror:0
[23]	train-merror:0.057229	test-merror:0
[24]	train-merror:0.057229

[185]	train-merror:0.01506	test-merror:0.055556
[186]	train-merror:0.01506	test-merror:0.055556
[187]	train-merror:0.01506	test-merror:0.055556
[188]	train-merror:0.01506	test-merror:0.055556
[189]	train-merror:0.01506	test-merror:0.055556
[190]	train-merror:0.01506	test-merror:0.055556
[191]	train-merror:0.01506	test-merror:0.055556
[192]	train-merror:0.01506	test-merror:0.055556
[193]	train-merror:0.01506	test-merror:0.055556
[194]	train-merror:0.009036	test-merror:0.055556
[195]	train-merror:0.009036	test-merror:0.055556
[196]	train-merror:0.009036	test-merror:0.055556
[197]	train-merror:0.009036	test-merror:0.055556
[198]	train-merror:0.009036	test-merror:0.055556
[199]	train-merror:0.009036	test-merror:0.055556
[200]	train-merror:0.006024	test-merror:0.055556
[201]	train-merror:0.006024	test-merror:0.055556
[202]	train-merror:0.006024	test-merror:0.055556
[203]	train-merror:0.006024	test-merror:0.055556
[204]	train-merror:0.006024	test-merror:0.055556
[205]	train-merror:0.006024	t

[353]	train-merror:0.003012	test-merror:0.055556
[354]	train-merror:0.003012	test-merror:0.055556
[355]	train-merror:0.003012	test-merror:0.055556
[356]	train-merror:0.003012	test-merror:0.055556
[357]	train-merror:0.003012	test-merror:0.055556
[358]	train-merror:0.003012	test-merror:0.055556
[359]	train-merror:0.003012	test-merror:0.055556
[360]	train-merror:0.003012	test-merror:0.055556
[361]	train-merror:0.003012	test-merror:0.055556
[362]	train-merror:0.003012	test-merror:0.055556
[363]	train-merror:0.003012	test-merror:0.055556
[364]	train-merror:0.003012	test-merror:0.055556
[365]	train-merror:0.003012	test-merror:0.055556
[366]	train-merror:0.003012	test-merror:0.055556
[367]	train-merror:0.003012	test-merror:0.055556
[368]	train-merror:0.003012	test-merror:0.055556
[369]	train-merror:0.003012	test-merror:0.055556
[370]	train-merror:0.003012	test-merror:0.055556
[371]	train-merror:0.003012	test-merror:0.055556
[372]	train-merror:0.003012	test-merror:0.055556
[373]	train-merror:0

#  SVM

In [144]:
model2 = SVC(C=1.0, kernel='rbf', degree=3, gamma='auto')
model2.fit(X_train,y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [153]:
pred2=model2.predict(X_test)
error2 = np.sum(pred2 != y_test) / y_test.shape[0]
print('Test error = {}'.format(error2))

Test error = 0.0
